
# Aim of the notebook


This notebook provides Python code for an extension of GMM methods to accomomodate model misspecification.  It uses relative entropy discrepancy to accomodate and measure the magnitude of the misspecification.  The standard GMM problem starts with 
$$ 
{\mathbb E} \left[ f(X_t, \theta) \right] = 0
$$
Instead we start with
$$
{\mathbb E} \left[ M f(X, \theta) \right] = 0
$$
for $M \ge 0$ and $E(M) = 1$. The random variable $M$ alters the probability measure.  We use $E( M \log M)$ to measure divergence in the altered probability and hence as a measure of misspecification. To implement these methods, we replace population moments sample analogs.


## Basic problem

For $\kappa > {\underline \kappa}$, a scalar function $h(X, \theta)$ and a set of admissible parameters $\Theta$, we solve:

\begin{equation*}
\min_{\theta \in \Theta} \min_M \mathbb{E} \left[M h(X,\theta)\right]
\end{equation*}
subject to
\begin{align*}
&\mathbb{E}\left[  M f(X, \theta) \right] = 0, \cr
&\mathbb{E} \left( M \right)  = 1, \cr
&\mathbb{E} \left( M \log M \right) \le \kappa
\end{align*}  

This problem has important special cases that interest us.




- Let $h(x,\theta)$ be plus or minus  indicator function for an event of interest independent of $\theta$.
By solving two problems we infer lower and upper bounds on probabilities of events of interest.  (minus the max of the negative of a function gives the min.)


- Partition $\theta = (\theta_1, \theta_2)$ where $\theta_1$ is the scalar parameter of interest.  Let 
$h(x,\theta) = \pm  \theta_1$.    By solving the two problems, we can construct upper and lower bounds on the parameter
$\theta_1.$


- Condition on $\theta_2$.  We repeat the previous problem and build the constraint into the construction of the  set $\Theta$.  Perform for alternative $\theta_2$'s and trace out a boundary of a set of admissible parameters.


## Dual problem


For computational purposes, we solve the dual problem after minimizing over $M$.  

\begin{equation*}
\min_{\theta \in \Theta}
\max_{\xi \ge 0, \lambda}    - \xi \log {\mathbb E} \left[ \exp\left( - {\frac 1 {\xi}} \left[ h(X, \theta) + \lambda \cdot f(X, \theta) \right] \right)\right]  -  \xi \kappa  
\end{equation*}

$\lambda$ and $\xi$ are multipliers on the moment condition and relative entropy constraints.  

## Lower bound on relative entropy

Under misspecification, there is lower bound on $\kappa$.  We compute this by solving:

\begin{equation*}
{\underline \kappa} \doteq  \min_{\theta \in \Theta} \min_M\mathbb{E}(M \log M)
\end{equation*}
subject to
\begin{align*}
&\mathbb{E}\left[  M f(X, \theta) \right] = 0, \cr
&\mathbb{E} \left( M \right) = 1 .
\end{align*}

Again this can be solved in a straightforward manner using a dual counterpart:

\begin{equation*}
{\underline \kappa} = \min_{\theta \in \Theta} \max_{\lambda} -\log \mathbb{E}\exp[-\lambda \cdot f(X,\theta)]
\end{equation*}

##### Han please organize the Python codes using this structure.


LPH stopped here.

## Dual problem when $h = h(\theta)$
When $h$ does not depend on $X$, we can take it out of the expectation. Then the dual problem becomes

\begin{equation*}
{\min_{\theta \in \Theta}}  h(\theta)+ 
\max_{\xi \ge 0} \max_{\lambda}  \left[- \xi \log {\mathbb E} \exp\left( - {\frac 1 {\xi}} \left[\lambda \cdot f(X, \theta) \right] \right)  -  \xi \kappa  \right]
\end{equation*}

Fix $\xi$ and maximize over $\lambda$, we have
\begin{equation*}
\min_{\theta \in \Theta}  h(\theta)+ 
\max_{\xi \ge 0} \xi\left(\kappa(\theta)-\kappa\right)
\end{equation*}

The primal form of this problem is
$$\min_{\theta \in \Theta} h(\theta)$$
*subject to*
$$\kappa(\theta) \leq \kappa$$

This form offers us computational advantages since computing relative entropy discrepancy is typically faster than computing the $\lambda,\xi$-maximization problem in the original dual problem.

## Python solver

The following Python class is designed to

- calculate the relative entropy discrepancy at any given $\theta$;


- find the lower bound on relative entropy;


- solve the basic problem with an arbitrary choice $h(X,\theta)$.

Examples come after the codes. 

In [1]:
# Load packages
import time
import itertools
import numpy as np
import pandas as pd
from numba import jit,float64
from scipy.optimize import minimize
from scipy.special import logsumexp

In [2]:
# Define numba accelerators
@jit
def objective_λ_numba(f,λ):
    # use "max trick"
    x = f@λ
    a = x.max()
    return np.log(np.sum(np.exp(x-a)))+a

@jit
def objective_gradient_λ_numba(f,λ):
    temp1 = f@λ
    temp2 = f*(np.exp(temp1.reshape((len(temp1),1)))/np.mean(np.exp(temp1)))
    temp3 = np.empty(temp2.shape[1])
    for i in range(temp2.shape[1]):
        temp3[i] = np.mean(temp2[:,i])
    return temp3

@jit
def objective_ξ_λ_numba(ξ_λ,f,h,κ,lower):
    ξ = ξ_λ[0]
    λ = ξ_λ[1:]
    temp1 = f@λ
    temp2 = -np.log(f.shape[0])+κ
    if lower == True:
        # use "max trick"
        x = -(temp1+h)/ξ
        a = x.max()
    else:
        # use "max trick"
        x = -(temp1-h)/ξ
        a = x.max()
    return (np.log(np.sum(np.exp(x-a)))+a+temp2)*ξ


# Define class
class Solver:
    def __init__(self,X,functionf):
        # Load data X and function f
        self.X = X
        self.functionf = functionf
        
        
    # Define the objective function of the maximization problem when calculating relative entropy
    def objective_λ(self,λ):
        return objective_λ_numba(self.f,λ)
        
    
    # Define the gradient of the objective function
    def objective_gradient_λ(self,λ):
        return objective_gradient_λ_numba(self.f,λ)
    
    
    # Maximize over λ
    def cal_divergence(self,
                       θ,
                       tol=1.0e-10,
                       maxiter=200):
        # Update f(X,θ)
        self.f = self.functionf(self.X,θ)
        
        # Create an initial point for λ
        initial_point = np.ones(self.f.shape[1])
        
        # Try L-BFGS-B, BFGS, CG sequentially
        # Switch the method if algorithm does not converge
        for method in ['L-BFGS-B','BFGS','CG']:
            # use SciPy solver
            model = minimize(self.objective_λ, 
                             initial_point, 
                             method=method,
                             jac=self.objective_gradient_λ,
                             tol=tol,
                             options={'maxiter': maxiter})
            
            # Check if algorithm converges
            if model.success == True:
                fun_value = -(model.fun-np.log(self.f.shape[0]))
                break
            else:
                fun_value = np.nan
            
        # Save results
        result = {'result':fun_value,
                'success':model.success,
                'message':model.message,
                'nit':model.nit}
        
        return result
    
    
    # Define the objective function of the ξ,λ-maximization problem
    # Take negative sign to solve the ξ,λ-minimization problem
    def objective_ξ_λ(self,ξ_λ):
        return objective_ξ_λ_numba(ξ_λ,self.f,self.h,self.κ,self.lower)
    
        
        
    # Maximize the objective over λ and ξ
    # Note: here κ is \kappa
    def maximize_ξ_λ(self,
                        κ,
                        θ,
                        functionh,
                        lower=True,
                        tol=1.0e-10,
                        maxiter=200):
        # Update parameters and function values
        self.κ = κ
        self.f = self.functionf(self.X,θ)
        self.lower = lower
        self.h = functionh(self.X,θ)
        self.tol = tol
        self.maxiter = maxiter
        
        # Check if the input θ satisfies the constraint
        div_res = self.cal_divergence(θ,tol=self.tol,maxiter=self.maxiter)
        
        if self.κ - div_res['result'] < 0:
            result = {'result':np.nan,
                   'success':False,
                   'message':'θ=' + str(θ) + ' is not feasible when κ=' + str(κ) + '...',
                   'nit':0}
        else:
            # Create an initial point
            initial_point = np.ones(self.f.shape[1]+1) 
            
            # Create bounds for the parameters
            bounds = []
            for i in range(self.f.shape[1]+1):
                if i==0:
                    bounds.append((0,None))
                else:
                    bounds.append((None,None))

            # Switch method if algorithm does not converge
            for method in ['L-BFGS-B','SLSQP']:
                model = minimize(self.objective_ξ_λ, 
                                 initial_point, 
                                 method=method,
                                 tol=tol,
                                 bounds=bounds,
                                 options={'maxiter':maxiter})

                # Check if algorithm converges
                if model.success == True:
                    fun_value = -model.fun
                    break
                else:
                    fun_value = np.nan
                          
            # Save optimization status
            result = {'result':fun_value,
                   'success':model.success,
                   'message':model.message,
                   'nit':model.nit}
        return result
    
    
    def divergence_lower_bound(self,
                               θ_bounds,
                               grid_size,
                               tol=1.0e-10,
                               maxiter=200):
        # Generate coordinate vectors: size*(dim of θ)
        θ_vec = np.array([np.linspace(θ_bounds[i][0],θ_bounds[i][1],grid_size) for i in range(len(θ_bounds))])
        # Generate coordinate meshgrid, size^(dim of θ)
        θ_grid = np.array(list(itertools.product(*θ_vec)))
        
        # Solve for each pair of θ
        result = []
        for θ in θ_grid:
            result.append(self.cal_divergence(θ,tol,maxiter))
        
        # Get divergence grid
        div_grid = np.array([result[i]['result'] for i in range(len(result))])
        # Find the minimal div
        div_min = np.nanmin(div_grid)
        
        # Find θ that minimizes div
        div_min_arg = np.nanargmin(div_grid)
        θ_min = θ_grid[div_min_arg]
        
        return {'div_min':div_min,
               'θ':θ_min}
    
        
    def expectation_bound(self,
                          κ,
                          functionh,
                          functionh_depends_on_x,
                          θ_independent,
                          θ_dependent,
                          grid_size,
                          tol=1.0e-10,
                          maxiter=200):
        # Generate coordinate vectors: size*(dim of θ_1)
        θ_vec = np.array([np.linspace(θ_dependent['bounds'][i][0],θ_dependent['bounds'][i][1],grid_size) \
                          for i in range(len(θ_dependent['bounds']))])
        # Generate coordinate meshgrid, size^(dim of θ_1)
        θ_grid = np.array(list(itertools.product(*θ_vec)))
        
        # Add independent θ to the meshgrid
        if θ_independent is not None:
            for i in range(len(θ_independent['positions'])):
                θ_grid = np.insert(θ_grid,θ_independent['positions'][i]-1,θ_independent['values'][i],axis=1)

        # Solve for each pair of θ
        result = []
        for θ in θ_grid:
            result.append(self.cal_divergence(θ,tol,maxiter))

        # Get divergence grid
        div_grid = np.array([result[i]['result'] for i in range(len(result))])
        
        # Find feasible θ grid and corresponding divergence grid
        div_grid[div_grid>κ] = np.nan
        θ_grid_feasible = θ_grid[~np.isnan(div_grid)]
        div_grid_feasible = div_grid[~np.isnan(div_grid)]
        
        # Check if κ is reasonable
        if len(θ_grid_feasible) == 0:
            print('No feasible region found...please increase κ...')
            return np.nan
        
        if functionh_depends_on_x == True:
            # Solve for each pair of θ
            result_exp = []
            for θ in θ_grid_feasible:
                result_lower = self.maximize_ξ_λ(κ,θ,functionh,True,tol,maxiter)
                result_upper = self.maximize_ξ_λ(κ,θ,functionh,False,tol,maxiter)
                result_exp.append([result_lower,result_upper])

            exp_low_grid = np.array([result_exp[i][0]['result'] for i in range(len(result_exp))])
            exp_up_grid = np.array([result_exp[i][1]['result'] for i in range(len(result_exp))])

            # Find the lower bound of the expectation
            exp_low = np.nanmin(exp_low_grid)
            # Find the upper bound of the expectation
            exp_up = -np.nanmin(exp_up_grid)

            # Find θ that minimizes the expectation
            exp_low_arg = np.nanargmin(exp_low_grid)
            θ_low = θ_grid_feasible[exp_low_arg]
            # Find θ that maximizes the expectation
            exp_up_arg = np.nanargmin(exp_up_grid)
            θ_up = θ_grid_feasible[exp_up_arg]

        else:
            # Create meshgrid for h(θ)
            h_θ_grid = np.array([functionh(X=None,θ=θ) for θ in θ_grid_feasible])
            # Find the smallest h(θ_dep) from the feasible region
            exp_low = np.nanmin(h_θ_grid)
            # Find the largest h(θ_dep) from the feasible region
            exp_up = np.nanmax(h_θ_grid)
            
            # Find θ that minimizes the expectation
            exp_low_arg = np.nanargmin(h_θ_grid)
            θ_low = θ_grid_feasible[exp_low_arg]
            # Find θ that maximizes the expectation
            exp_up_arg = np.nanargmax(h_θ_grid)
            θ_up = θ_grid_feasible[exp_up_arg]
        
        return {'exp_low':exp_low,
                'exp_up':exp_up,
                'θ_low':θ_low,
                'θ_up':θ_up}

### Example 1. calculate relative entropy discrepancy

Suppose $X$ include five variables: $G_{t,t+1},R_{f,t+1},R_{m,t+1}^e,R_{SMB,t+1}^e,R_{HML,t+1}^e$.

Let $f(X,\theta)$ be

$$ f(X_{t+1},\theta) = 
\left[ \begin{array}{c} 
\delta \left( G_{t,t+1} \right)^{-\gamma} R_{f,t+1}  - 1  \\ 
\delta \left( G_{t,t+1} \right)^{-\gamma} R_{m,t+1}^e \\
\delta \left( G_{t,t+1} \right)^{-\gamma} R_{SMB,t+1}^e \\
\delta \left( G_{t,t+1} \right)^{-\gamma} R_{HML,t+1}^e
\end{array} \right] $$

The relative entropy discrepancy is given by
$$\kappa(\theta) = \displaystyle \max_{\lambda} -\log \mathbb{E}\exp[-\lambda \cdot f(X,\theta)]$$

Since the objective is concave in $\lambda$, the solver uses convex optimization algorithm [L-BFGS-B](http://sepwww.stanford.edu/data/media/public/docs/sep117/antoine1/paper_html/node6.html) (or [BFGS](https://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm), [CG](https://en.wikipedia.org/wiki/Conjugate_gradient_method) if it does not converge) to solve $\kappa(\theta)$.

In [3]:
# Load dataset 
data = pd.read_csv('CleanDataQuarterly.csv',index_col=0)
X = np.array(data)

# Define f(X,θ), using numba accelerator
@jit(nopython=True)
def functionf(X,θ):
    vec = np.power(X[:,0:1],-θ[1])
    M1 = X[:,1:] * vec
    Y = M1 * θ[0] - np.array([1,0,0,0])
    return Y

In [4]:
time_start = time.time() # Count time
solver = Solver(X,functionf) # Initialize solver
result = solver.cal_divergence(θ=np.array([0.95,15.0]), # θ at which κ(θ) being calculated
                               tol=1.0e-5, # Tolerance level
                               maxiter=200) # Maximal # of iterations

# Print results
print('The relative entropy divergence is: '+str(result['result'])+'.')
print("--- %s seconds ---" % (round(time.time()-time_start,4)))

The relative entropy divergence is: 0.5856825399020096.
--- 1.5409 seconds ---


### Example 2. find lower bound on relative entropy

The lower bound can be solved by

$$\underline{\kappa} = \displaystyle \min_{\theta \in \Theta}\kappa(\theta)$$

$\kappa(\theta)$ is not necessarily convex in $\theta$. The solver uses Grid Search approach to find $\underline{\kappa}$.

In [5]:
time_start = time.time() # Count time
solver = Solver(X,functionf) # Initialize solver
result = solver.divergence_lower_bound(θ_bounds=[(0.95,1.0),(5.0,35.0)], # Bounds of θ
                                       grid_size=100, # Specify # of grid points in each parameter dimension
                                       tol=1.0e-5, # Tolerance level
                                       maxiter=200) # Maximal # of iterations

# Print results
print('The minimal relative entropy is: '+str(result['div_min']) +', achieved at θ='+str(result['θ']))
print("--- %s seconds ---" % (round(time.time()-time_start,4)))

The minimal relative entropy is: 0.15264768751860647, achieved at θ=[1. 5.]
--- 3.4776 seconds ---


### Example 3. find lower/upper bounds on expectation

The lower bound on expectation of $h(X,\theta)$ is given by
\begin{equation*}
{\min_{\theta \in \Theta}} 
\max_{\xi \ge 0, \lambda}    - \xi \log {\mathbb E} \left[ \exp\left( - {\frac 1 {\xi}} \left[ h(X, \theta) + \lambda \cdot f(X, \theta) \right] \right)\right]  -  \xi \kappa  
\end{equation*}

Since the objective is concave in $(\xi, \lambda)$, the solver will first use convex optimization algorithms [L-BFGS-B](http://sepwww.stanford.edu/data/media/public/docs/sep117/antoine1/paper_html/node6.html) or [SLSQP](http://degenerateconic.com/slsqp/) to solve

\begin{equation*}
\max_{\xi \ge 0, \lambda}    - \xi \log {\mathbb E} \left[ \exp\left( - {\frac 1 {\xi}} \left[ h(X, \theta) + \lambda \cdot f(X, \theta) \right] \right)\right]  -  \xi \kappa  
\end{equation*}

for each pair of $\theta \in \Theta$. Then it uses Grid Search approach to minimize over $\theta$. The upper bound problem can be solved in the same way by replacing $h$ with $-h$.

The choice of $h(X,\theta)$ can be arbitrary. Below we show three important special cases.

#### Case 1. $h(X,\theta)$ is an indicator function of $X$

$$h(X_{t+1}) = \mathbb{1} \{R_{m,t+1}^e>0\}$$

In [ ]:
# Define h(X,θ), using numba accelerator
@jit
def functionh(X,θ):
    vec = X[:,2].copy()
    vec[vec>0] = 1
    vec[vec<0] = 0
    return vec

np.seterr(divide='ignore', invalid='ignore')
time_start = time.time() # Count time
model = Solver(X,functionf) # Initialize solver
result = model.expectation_bound(κ=1.0, # Threshold κ
                                 functionh=functionh, # Function defined above
                                 functionh_depends_on_x=True, # True if functionh depends on x
                                 θ_independent=None, # Not condition on any θ_i
                                 θ_dependent={'positions':[1,2], # Target θ=(θ_1,θ_2)
                                             'bounds':[(0.95,1.0),(5.0,35.0)]}, # Bounds of target θ 
                                 grid_size=100, # Specify grid size
                                 tol=1.0e-10, # Tolerance level
                                 maxiter=200) # Maximal # of iterations

# Print results
print('The lower bound for the expectation is: '+str(result['exp_low']) +', achieved at θ='+str(result['θ_low']))
print('The upper bound for the expectation is: '+str(result['exp_up']) + ', achieved at θ='+str(result['θ_up']))
print("--- %s seconds ---" % (round(time.time()-time_start,4)))

#### Case 2. $h(X,\theta) = \theta_1$

$$h(X,\theta) = \theta_1$$

where $\theta_1$ is a scalar parameter of our interest.

In [ ]:
# Define h(X,θ), using numba accelerator
@jit
def functionh(X,θ):
    return θ[0]

time_start = time.time() # Count time
model = Solver(X,functionf) # Initialize solver
result = model.expectation_bound(κ=1.0, # Threshold κ
                                 functionh=functionh, # Function defined above
                                 functionh_depends_on_x=False, # True if functionh depends on x
                                 θ_independent=None, # Not condition on any θ_i
                                 θ_dependent={'positions':[1,2], # Target θ=(θ_1,θ_2)
                                             'bounds':[(0.95,1.0),(5.0,35.0)]}, # Bounds of target θ 
                                 grid_size=100, # Specify grid size
                                 tol=1.0e-5, # Tolerance level
                                 maxiter=200) # Maximal # of iterations

# Print results
print('The lower bound for the expectation is: '+str(result['exp_low']) +', achieved at θ='+str(result['θ_low']))
print('The upper bound for the expectation is: '+str(result['exp_up']) + ', achieved at θ='+str(result['θ_up']))
print("--- %s seconds ---" % (round(time.time()-time_start,4)))

#### Case 3. $h(X,\theta) = \theta_1$, condition on $\theta_2$

$$h(X,\theta) = \theta_1$$

where $\theta_1$ is a scalar parameter of our interest.

In [ ]:
# Define h(X,θ)=θ_1, using numba accelerator
@jit
def functionh(X,θ):
    return θ[0]

time_start = time.time() # Count time
solver = Solver(X,functionf) # Initialize solver
result = solver.expectation_bound(κ=1.0, # Threshold κ
                                  functionh=functionh, # Function defined above
                                  functionh_depends_on_x=False, # True if functionh depends on x
                                  θ_independent={'positions':[2], # Condition on θ_2
                                                 'values':[15.0]}, # Value of θ_2
                                  θ_dependent={'positions':[1], # Target θ=(θ_1)
                                               'bounds':[(0.95,1.0)]}, # Bounds of target θ
                                  grid_size=100, # Specify grid size
                                  tol=1.0e-5, # Tolerance level
                                  maxiter=200) # Maximal # of iterations

# Print results
print('The lower bound for the expectation is: '+str(result['exp_low']) +', achieved at θ='+str(result['θ_low']))
print('The upper bound for the expectation is: '+str(result['exp_up']) + ', achieved at θ='+str(result['θ_up']))
print("--- %s seconds ---" % (round(time.time()-time_start,4)))